 closing_bottle         drinking       fastening_seat_belt                          opening_bottle         placing_an_object              putting_on_jacket       sitting_still                 talking_on_phone           writing
 closing_door_inside    eating         fetching_an_object                           opening_door_inside    preparing_food                 putting_on_sunglasses   taking_laptop_from_backpack   unfastening_seat_belt
 closing_door_outside   entering_car   interacting_with_phone                       opening_door_outside   pressing_automation_button     reading_magazine        taking_off_jacket             using_multimedia_display
 closing_laptop         exiting_car   'looking_or_moving_around (e.g. searching)'   opening_laptop         putting_laptop_into_backpack   reading_newspaper       taking_off_sunglasses         working_on_laptop

In [1]:
train_dir = "/net/polaris/storage/deeplearning/sur_data/kinect_ir_daa/split_2/train"
val_dir = "/net/polaris/storage/deeplearning/sur_data/kinect_ir_daa/split_2/val"
test_dir = "/net/polaris/storage/deeplearning/sur_data/kinect_ir_daa/split_2/test"

In [2]:
import os

def count_images(directory):
    extensions = ('.jpg', '.jpeg', '.png', '.gif')  # Add or remove extensions as needed
    num_images = 0
    for root, dirs, files in os.walk(directory):
        num_images += sum(1 for file in files if file.lower().endswith(extensions))
    return num_images

In [3]:
print(f"Number of images in KIR Split_2 train directory: {count_images(train_dir)}")
print(f"Number of images in KIR Split_2 val directory: {count_images(val_dir)}")
print(f"Number of images in KIR Split_2 test directory: {count_images(test_dir)}")

Number of images in KIR Split_2 train directory: 299647
Number of images in KIR Split_2 val directory: 61803
Number of images in KIR Split_2 test directory: 99467


In [4]:
new_train_dir = "/net/polaris/storage/deeplearning/sur_data/binary_kinect_ir_daa/split_2/train"
new_val_dir = "/net/polaris/storage/deeplearning/sur_data/binary_kinect_ir_daa/split_2/val"
new_test_dir = "/net/polaris/storage/deeplearning/sur_data/binary_kinect_ir_daa/split_2/test"

In [5]:
import os
import pathlib
from concurrent.futures import ProcessPoolExecutor
from shutil import copy2
from typing import Tuple

class BinaryImageOrganizer:
    def __init__(self, source_dir: str, target_dir: str) -> None:
        self.source_dir = source_dir
        self.target_dir = target_dir
        self.non_distracted_classes = {'sitting_still', 'entering_car', 'exiting_car'}
        self.setup_directories()
        # Initialize counters for image filenames
        self.image_counters = {
            '_non_distracted': 0,
            'distracted': 0
        }

    def setup_directories(self):
        os.makedirs(self.target_dir, exist_ok=True)
        os.makedirs(os.path.join(self.target_dir, '_non_distracted'), exist_ok=True)
        os.makedirs(os.path.join(self.target_dir, 'distracted'), exist_ok=True)

    def is_non_distracted(self, class_name: str) -> bool:
        return class_name in self.non_distracted_classes

    def process_image(self, image_path: pathlib.Path) -> Tuple[str, str]:
        class_name = image_path.parent.parent.name
        binary_folder = '_non_distracted' if self.is_non_distracted(class_name) else 'distracted'
        target_folder = os.path.join(self.target_dir, binary_folder)
        # Increment the counter for the appropriate class and format the filename
        self.image_counters[binary_folder] += 1
        new_filename = f"img_{self.image_counters[binary_folder]:06d}.png"
        return str(image_path), os.path.join(target_folder, new_filename)

    def copy_images(self, image_paths):
        for src, dest in image_paths:
            try:
                copy2(src, dest)
            except Exception as e:
                print(f"Failed to copy {src} to {dest}: {e}")

    def organize_images(self):
        paths = list(pathlib.Path(self.source_dir).glob("*/*/*.png"))
        image_tasks = [self.process_image(path) for path in paths]
        
        # Split tasks for multiprocessing
        chunk_size = len(image_tasks) // os.cpu_count() + 1
        chunks = [image_tasks[i:i + chunk_size] for i in range(0, len(image_tasks), chunk_size)]

        with ProcessPoolExecutor() as executor:
            executor.map(self.copy_images, chunks)


In [6]:
# Split 2 RGB : Validation dataset
organizer = BinaryImageOrganizer(
    source_dir= val_dir,
    target_dir= new_val_dir
)
organizer.organize_images()

In [7]:
# Split 2 RGB : Train dataset
organizer = BinaryImageOrganizer(
    source_dir= train_dir,
    target_dir= new_train_dir
)
organizer.organize_images()

In [8]:
# Split 2 RGB : Test dataset
organizer = BinaryImageOrganizer(
    source_dir= test_dir,
    target_dir= new_test_dir
)
organizer.organize_images()

In [9]:
import torch
import torchvision
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=new_train_dir)
val_dataset = ImageFolder(root=new_val_dir)
test_dataset = ImageFolder(root=new_test_dir)

In [10]:
print(f"The length of the Train dataset split_2 KIR is: {len(train_dataset)}")
print(f"The length of the Validation dataset split_2 KIR is: {len(val_dataset)}")
print(f"The length of the Test dataset split_2 KIR is: {len(test_dataset)}")

The length of the Train dataset split_2 KIR is: 299647
The length of the Validation dataset split_2 KIR is: 61803
The length of the Test dataset split_2 KIR is: 99467


In [62]:
train_dataset.classes

['_non_distracted', 'distracted']

In [63]:
train_dataset.class_to_idx

{'_non_distracted': 0, 'distracted': 1}